# 2. Coordinate Transformations

## Working with Units

In [2]:
import astropy.units as u
from astroquery.gaia import Gaia

In [3]:
angle = 10 * u.degree
type(angle)

astropy.units.quantity.Quantity

In [4]:
angle

<Quantity 10. deg>

In [5]:
angle_arcmin = angle.to(u.arcmin)
angle_arcmin

<Quantity 600. arcmin>

In [6]:
angle + 30 * u.arcmin

<Quantity 10.5 deg>

### Exercise

In [20]:
radius = 5 * u.arcmin
radius = radius.to(u.degree)

## Selecting a Region

In [25]:
cone_query_base = """SELECT 
TOP 10 
source_id
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT(ra, dec),
  CIRCLE(88.8, 7.4, {radius}))
"""

cone_query = cone_query_base.format(radius=radius.value)

print(cone_query)

SELECT 
TOP 10 
source_id
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT(ra, dec),
  CIRCLE(88.8, 7.4, 0.08333333333333333))



In [26]:
cone_job = Gaia.launch_job(cone_query)
cone_job

In [29]:
cone_results = cone_job.get_results()
display(cone_results)

source_id
int64
3322773965056065536
3322773758899157120
3322774068134271104
3322773930696320512
3322774377374425728
3322773724537891456
3322773724537891328
3322773930696321792
3322773724537890944


### Exercise

In [30]:
cone_query_base = """SELECT 
COUNT(source_id)
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT(ra, dec),
  CIRCLE(88.8, 7.4, {radius}))
"""

cone_query = cone_query_base.format(radius=radius.value)

print(cone_query)

SELECT 
COUNT(source_id)
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT(ra, dec),
  CIRCLE(88.8, 7.4, 0.08333333333333333))



In [31]:
cone_job = Gaia.launch_job(cone_query)
cone_results = cone_job.get_results()
display(cone_results)

COUNT
int64
594


## Transforming Coordinates

In [39]:
from astropy.coordinates import SkyCoord
from gala.coordinates import GD1Koposov10

In [40]:
ra = 88.8 * u.degree
dec = 7.4 * u.degree
coord_icrs = SkyCoord(ra=ra, dec=dec, frame='icrs')

coord_icrs

<SkyCoord (ICRS): (ra, dec) in deg
    (88.8, 7.4)>

In [41]:
coord_galactic = coord_icrs.transform_to('galactic')
coord_galactic

<SkyCoord (Galactic): (l, b) in deg
    (199.79693102, -8.95591653)>

In [50]:
gd1_frame = GD1Koposov10
coord_gd1 = coord_icrs.transform_to(gd1_frame)
display(coord_gd1)

<SkyCoord (GD1Koposov10): (phi1, phi2) in deg
    (-94.97222038, 34.5813813)>

### Exercise

In [56]:
coord_gd_1 = SkyCoord(0 * u.degree, 0 * u.degree, frame=gd1_frame)
coord_gd_1 =  coord_gd_1.transform_to('icrs')
display(coord_gd_1)

<SkyCoord (ICRS): (ra, dec) in deg
    (200., 59.4504341)>

## Selecting a Rectangle

define corners of rectangle

In [57]:
phi1_min = -55 * u.degree 
phi1_max = -45 * u.degree
phi2_min = -8 * u.degree
phi2_max = 4 * u.degree

In [59]:
def make_rectangle(x1, x2, y1, y2):
    xs = [x1, x1, x2, x2, x1]
    ys = [y1, y2, y2, y1, y1]
    return xs, ys

In [60]:
phi1_rect, phi2_rect = make_rectangle(phi1_min, phi1_max, phi2_min, phi2_max)

convert from gd-1 frame to ICRS

In [61]:
corners = SkyCoord(phi1=phi1_rect, phi2=phi2_rect, frame=gd1_frame)
display(corners)

<SkyCoord (GD1Koposov10): (phi1, phi2) in deg
    [(-55., -8.), (-55.,  4.), (-45.,  4.), (-45., -8.), (-55., -8.)]>

rectangle in gd-1 frame becomes polygon is ICRS coordinates

In [63]:
corners_icrs = corners.transform_to('icrs')
display(corners_icrs)

<SkyCoord (ICRS): (ra, dec) in deg
    [(146.27533314, 19.26190982), (135.42163944, 25.87738723),
     (141.60264825, 34.3048303 ), (152.81671045, 27.13611254),
     (146.27533314, 19.26190982)]>

function to convert coordinates into format readable by query

In [73]:
def skycoord_to_string(skycoord):
    corners_list_str = skycoord.to_string()
    corners_single_str = ' '.join(corners_list_str)
    return corners_single_str.replace(' ', ', ')

In [74]:
sky_point_list = skycoord_to_string(corners_icrs)
sky_point_list

'146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619'

make query

In [75]:
columns = 'source_id, ra, dec, pmra, pmdec, parallax'

In [85]:
polygon_query_base = """SELECT
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT(ra, dec), 
                   POLYGON({sky_point_list}))
"""

In [86]:
polygon_query = polygon_query_base.format(columns=columns, sky_point_list=sky_point_list)
print(polygon_query)

SELECT
source_id, ra, dec, pmra, pmdec, parallax
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT(ra, dec), 
                   POLYGON(146.275, 19.2619, 135.422, 25.8774, 141.603, 34.3048, 152.817, 27.1361, 146.275, 19.2619))



In [89]:
polygon_job = Gaia.launch_job_async(polygon_query)
print(polygon_job)

INFO: Query finished. [astroquery.utils.tap.core]
<Table length=140339>
   name    dtype    unit                              description                            
--------- ------- -------- ------------------------------------------------------------------
source_id   int64          Unique source identifier (unique within a particular Data Release)
       ra float64      deg                                                    Right ascension
      dec float64      deg                                                        Declination
     pmra float64 mas / yr                         Proper motion in right ascension direction
    pmdec float64 mas / yr                             Proper motion in declination direction
 parallax float64      mas                                                           Parallax
Jobid: 1702867965761O
Phase: COMPLETED
Owner: None
Output file: async_20231217215245.vot
Results: None


In [92]:
polygon_results = polygon_job.get_results()
print(f'# of samples: {len(polygon_results)}')

# of samples: 140339


In [95]:
with open('gd1_results.fits') as f:
    polygon_results.write('gd1_results.fits', overwrite=True)